<a href="https://colab.research.google.com/github/tzhsu211/CVS/blob/main/Optimizer0207.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.6/233.6 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 7.6 MB/s eta 0:00:00


In [2]:
import torch
import pandas as pd
import numpy as np
from transformers import BertTokenizerFast, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import optuna
from accelerate import Accelerator
from datasets import Dataset
from matplotlib import pyplot as plt

In [3]:
torch.cuda.is_available()

True

In [4]:
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('./cvs_products.csv')
df1 = df.drop(['product', 'store', 'link', 'CVS'], axis = 1)
scaler = StandardScaler()
df1['rating_standard'] = scaler.fit_transform(df['rating'].values.reshape(-1, 1))
X_train, X_test, y_train, y_test = train_test_split(df1['review'], df1['rating_standard'], test_size=0.1, random_state=42)
train_df = pd.DataFrame({'text': X_train, 'label': y_train.astype(float)})
test_df = pd.DataFrame({'text': X_test, 'label': y_test.astype(float)})

train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

In [6]:
def preprocess_function(examples, tokenizer):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=256)

ckiplab_bert_tokenizer = BertTokenizerFast.from_pretrained('ckiplab/bert-base-chinese-ws')
ckiplab_bert_train_dataset = train_dataset.map(lambda x: preprocess_function(x, ckiplab_bert_tokenizer), batched=True)
ckiplab_bert_test_dataset = test_dataset.map(lambda x: preprocess_function(x, ckiplab_bert_tokenizer), batched=True)

tokenizer_config.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/804 [00:00<?, ?B/s]

Map:   0%|          | 0/15602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1734 [00:00<?, ? examples/s]

In [7]:
def compute_metrics(p):
    preds, labels = p
    preds = preds.squeeze()
    mse = mean_squared_error(labels, preds)
    r2 = r2_score(labels, preds)
    return {"mse": mse, "r2": r2}

In [8]:
def train_model(model, tokenizer_model, model_name, train_dataset = train_dataset, test_dataset = test_dataset, num_train_epochs=10, batch_size=32):

    tokenizer = BertTokenizerFast.from_pretrained(tokenizer_model)

    train_dataset = train_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)
    test_dataset = test_dataset.map(lambda x: preprocess_function(x, tokenizer), batched=True)

    # 設定 TrainingArguments
    training_args = TrainingArguments(
        output_dir=f'./results/{model_name}',
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir=f'./logs/{model_name}',
        logging_steps=100,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        fp16=True
    )

    # 初始化 Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics, # return mse & r2
        callbacks = [EarlyStoppingCallback(early_stopping_patience=5)] # early stopping
    )

    trainer.train()

    return trainer


In [9]:
model_bert_base = AutoModelForSequenceClassification.from_pretrained(
    'google-bert/bert-base-chinese',
    num_labels=1,
    ignore_mismatched_sizes=True
)
model_bert_base.classifier = torch.nn.Linear(model_bert_base.config.hidden_size, 1)

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
train_model(model_bert_base, tokenizer_model= 'google-bert/bert-base-chinese', model_name = 'google_bert_bas1', num_train_epochs = 15, batch_size = 16)

Map:   0%|          | 0/15602 [00:00<?, ? examples/s]

Map:   0%|          | 0/1734 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Mse,R2
1,0.133900,0.603426,0.603426,0.459336
2,0.119900,0.581647,0.581647,0.478849
3,0.114300,0.517131,0.517131,0.536655
4,0.094900,0.525722,0.525722,0.528958
5,0.074800,0.522846,0.522846,0.531535
6,0.061600,0.564807,0.564807,0.493938
7,0.048700,0.543252,0.543252,0.513251
8,0.040400,0.500903,0.500903,0.551196
9,0.031600,0.513143,0.513143,0.540229
10,0.031100,0.509176,0.509176,0.543783


In [14]:
checkpoint_path = './ckiplab_bert'
ckiplab_bert_model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path, config=f'{checkpoint_path}/config.json')


In [15]:
def objective(trial, model, model_name, train_dataset, eval_dataset):

    # Define hyperparameters to tune
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-6, 1e-3)
    dropout = trial.suggest_uniform('dropout', 0.1, 0.5)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32, 64])
    weight_decay = trial.suggest_loguniform('weight_decay', 0.01, 0.1)

    # Set dropout in model config (dropout is not set in training_args)
    model.config.attention_probs_dropout_prob = dropout
    model.config.hidden_dropout_prob = dropout

    # Set training_args based on fine-tune hyperparameters
    training_args = TrainingArguments(
        output_dir=f'./results/{model_name}_trial_{trial.number}',
        logging_dir=f'./logs/{model_name}_trial_{trial.number}',
        logging_steps=100,
        save_strategy='epoch',
        evaluation_strategy='epoch',
        warmup_steps=300,
        num_train_epochs=5,
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        weight_decay=weight_decay,
        load_best_model_at_end=True,
        fp16=True,
        seed=42,
    )

    accelerator = Accelerator(mixed_precision="fp16")

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    trainer.train()

    # Evaluation
    eval_result = trainer.evaluate()

    return eval_result['eval_loss']


In [ ]:
study = optuna.create_study(
    study_name="study0207",
    direction='minimize',
    storage='sqlite:////content/optuna_db/study.db',
    load_if_exists=True
)

study.optimize(
    lambda trial: objective(
        trial,
        ckiplab_bert_model,
        "ckiplab-bert-base_optuna",
        ckiplab_bert_train_dataset,
        ckiplab_bert_test_dataset
    ),
    n_trials=10
)

[I 2025-02-07 10:46:57,845] Using an existing study with name 'study0207' instead of creating a new one.


Epoch,Training Loss,Validation Loss,Mse,R2
1,0.410500,0.562798,0.562798,0.425060
2,0.234100,0.539470,0.539470,0.448892
3,0.187000,0.520332,0.520332,0.468442
4,0.173300,0.520930,0.520930,0.467831
5,0.162100,0.513712,0.513712,0.475206


[I 2025-02-07 10:59:52,359] Trial 1 finished with value: 0.5137115716934204 and parameters: {'learning_rate': 1.8425109934864179e-06, 'dropout': 0.49620406576132814, 'batch_size': 64, 'weight_decay': 0.01581627078804455}. Best is trial 1 with value: 0.5137115716934204.


Epoch,Training Loss,Validation Loss,Mse,R2
1,0.143400,0.506918,0.506918,0.482145
2,0.129400,0.519981,0.519981,0.468801
3,0.110300,0.495671,0.495671,0.493635
4,0.102500,0.512180,0.512180,0.476770
5,0.082500,0.495858,0.495858,0.493444


[I 2025-02-07 11:12:07,145] Trial 2 finished with value: 0.4956710636615753 and parameters: {'learning_rate': 5.054410702702853e-06, 'dropout': 0.4357289854588492, 'batch_size': 16, 'weight_decay': 0.05768048078585931}. Best is trial 2 with value: 0.4956710636615753.


...

[I 2025-02-07 11:32:28,202] Trial 4 finished with value: 0.4849853217601776 and parameters: {'learning_rate': 0.00011516903788767414, 'dropout': 0.1587322161082344, 'batch_size': 32, 'weight_decay': 0.03909910497729743}. Best is trial 4 with value: 0.4849853217601776.


Epoch,Training Loss,Validation Loss,Mse,R2
1,0.051500,0.486736,0.486736,0.502764
2,0.051000,0.487282,0.487282,0.502205
3,0.045800,0.506795,0.506795,0.482272


[I 2025-02-07 11:37:44,111] Trial 5 finished with value: 0.4867355525493622 and parameters: {'learning_rate': 1.2164779313818263e-05, 'dropout': 0.4994502311555332, 'batch_size': 64, 'weight_decay': 0.07121529801477762}. Best is trial 4 with value: 0.4849853217601776.


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss
